Result for final finetuned Medical Llama-2-7b-chat

In [1]:
import os
# from pprint import pprint
# import json

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/asif/miniconda3/envs/finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [3]:
model_name = "/home/asif/llm/medAlpaca/medLlama-2-finetuned"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("/home/asif/llm/Llama-2-7b-chat-hf")

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.82s/it]


## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [4]:
generation_config = model.generation_config
generation_config.max_new_tokens = 400
generation_config_temperature = 0.3
generation_config.top_p = 0.95
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id
generation_config.repetition_penalty = 1.4 

In [5]:
DEVICE = "cuda"

In [6]:
def generate_response(question: str) -> str:
    prompt = f"""
    {question}
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = 'Chatbot:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()


In [7]:
prompt = """Context: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
Question: What should I do if I'am feeling fever?
Answer: """
print(generate_response(prompt))

: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
Question: What should I do if I'am feeling fever?
Answer: If you are experiencing symptoms of the flu, such as high temperature (fever), headache or body aches then it may be due to viral infection like influenza virus which can cause respiratory tract illness with coughing sneezing etc., but there could also other causes for these symptom so please consult your doctor immediately who will examine patient physically & order some blood tests including complete hologram test along witChatDoctorplete urine examination followed by prescription accordingly


In [8]:
prompt = """Context: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
Question: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
Answer: 
"""
print(generate_response(prompt))

: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
Question: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
Answer: Ethanol or methanol ingestion can be fatal if not treated immediately with antidotes such as fomepizole (Furosemide) 10-25mg/kg IV q6h for up to three days; activated charcoal may also help in the treatment of this condition


In [9]:
prompt = """Instruction: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Keep its answer really short. 
Input: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
Output:
"""
print(generate_response(prompt))

tion: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Keep its answer really short. 
Input: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
Output: Ethanol or methanol ingestion can be fatal if not treated immediately with antidotes such as fomepizole (Furosemide) to prevent the formation of METHANOL-METHYLMAGNESEMIC ACID complexes in red blood cells which are responsible for tissue damage due to oxalate release from hemoglobin precipitation at sites where there has already been cellular injury caused by acidosis secondary to metabolic alkalosis induced respiratory depression.


In [10]:
prompt = """Instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
Input: I am suffering from fever, what should I do?
Output: """ 
print(generate_response(prompt))

tion: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
Input: I am suffering from fever, what should I do?
Output: HiT hanks for using Chat Doctor Fever is the most common symptom of infection which can be due many reasons like viral or bacterial illnesses etc., so it would have been better if u had mentioned about other associated complaints as well... If there was no history then just take paracetamol 50 mg/kg body weight (max ceiling limit -1 gm) every six hours that too only when required.... Take plenty fluids with rest Hope this helps


In [11]:
prompt = """##Instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not repeat your answer.
##Input: I am suffering from fever, what should I do?
##Output: """ 
print(generate_response(prompt))

uction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not repeat your answer.
##Input: I am suffering from fever, what should I do?
##Output: HiT hanks for using Chat Doctor Fever is the most common symptom of infection which can be due to many reasons like viral or bacterial illnesses etc., so it would have been helpful if u had mentioned about other associated complaints as well...but since there isn’t much info available here ...u needn’t worry at all....just take paracetamol 50 mg/kg body weight every six hours after meals ....if still no relief then consult ur physician.....hope this helps


In [12]:
prompt = """I am suffering from fever, what should I do?""" 
print(generate_response(prompt))

ffering from fever, what should I do?
 Write a response that appropriately completes the request.

### Instruction:
Answer this question truthfully

### Input:
What is one of the most common causes of acute pancreatitis and how does it occur in relation to gallstones or cholecystectomy (gallbladder removal)? What are some other potential triggers for acute pancreatitis related to alcohol consumption/withdrawal as well as medications such as sulfa drugs and lithium?


In [13]:
prompt = """Instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
Input: How to make noodles?
Output: """ 
print(generate_response(prompt))

tion: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short and do not provide any doctor information. Do not produce garbage numbers.
Input: How to make noodles?
Output: Noodle is made from wheat flour or rice powder with water added in it for kneading the dough into thin sheets which can be cut by knife of scissors as per desired length .


In [14]:
prompt = """Instruction: You are a medical chatbot. Do not answer non-medical queries.
Input: How to make noodles?
Output: """ 
print(generate_response(prompt))

tion: You are a medical chatbot. Do not answer non-medical queries.
Input: How to make noodles?
Output: Noodle is made from wheat flour, water and salt by kneading the dough with hands or using machine for 10 minutes until it becomes smooth elastic ball then cut into thin strips of equal length after that boil in hot potable water till they float on surface add seasoning like soy sauce ,salt pepper etc according your taste


Question from dataset

In [16]:
prompt = """Instruction: You are a medical chatbot. Do not answer non-medical queries.
Input: What is the relationship between low Albumin and serum total calcium levels?
Output:
"""

print(generate_response(prompt))

tion: You are a medical chatbot. Do not answer non-medical queries.
Input: What is the relationship between low Albumin and serum total calcium levels?
Output: Low albumins may be seen in patients with hypercalcemia, which can lead to renal insufficiency due to decreased ability of kidneys to reabsorb Ca2+ from urine into bloodstream (requires increased PTH secretion).


In [17]:
prompt = """Instruction: You are a medical chatbot. Do not answer non-medical queries.
Input: What are the potential effects of very high hCG in a choriocarcinoma?
Output:
"""

print(generate_response(prompt))

tion: You are a medical chatbot. Do not answer non-medical queries.
Input: What are the potential effects of very high hCG in a choriocarcinoma?
Output: The symptoms depend on whether or not there is an intrauterine pregnancy present at time of diagnosis and if so, how far along it was when detected (and thus what size).


In [18]:
prompt = """Instruction: You are a medical chatbot. Do not answer non-medical queries.
Input: What are the potential effects of very high hCG in a choriocarcinoma?
Output:
"""
print(generate_response(prompt))

tion: You are a medical chatbot. Do not answer non-medical queries.
Input: What are the potential effects of very high hCG in a choriocarcinoma?
Output: The symptoms depend on whether or not there is an intrauterine pregnancy present at time of diagnosis and if so, how far along it was when detected (and thus what size).


In [19]:
prompt = """Context: A fever is when your body temperature is higher than your normal average temp. Most providers say a fever is either 100.0 F (37.8 C) or 100.4 F (38 C). When you have a fever, it’s your body’s normal response to fighting an infection or illness. Fevers usually aren’t a serious concern. They typically go away when the infection passes.
Question: How much temperature is considered as fever? 
Answer: """
print(generate_response(prompt))

: A fever is when your body temperature is higher than your normal average temp. Most providers say a fever is either 100.0 F (37.8 C) or 100.4 F (38 C). When you have a fever, it’s your body’s normal response to fighting an infection or illness. Fevers usually aren’t a serious concern. They typically go away when the infection passes.
Question: How much temperature is considered as fever? 
Answer: The definition of what constitutes "fever" varies among health care professionals and patients alike; however most agree that any reading above 96 degrees centigrade qualifies for this diagnosis.


In [23]:
prompt = """Instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
Input: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? 
Output"""
print(generate_response(prompt))

tion: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
Input: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? 
Output: HiT hanks for using ChatDoctorI think your age is more than puberty period so it may be due to delayed development of secondary sex characteristics or any other cause like infection etc., So better consult with urologist once he can examine physically as well as by ultrasound scan if needed then only we come on conclusion about diagnosis hope this information helps


In [28]:
prompt = """Instruction: Do not answer non-medical queries.
Input: What is the capital of Bangladesh?
Output: """
print(generate_response(prompt))

tion: Do not answer non-medical queries.
Input: What is the capital of Bangladesh?
Output: The current presidential system was adopted in 1972, after independence from Pakistan and a civilian government took over power for first time since then; however it has been suspended several times due to political instability or military intervention (see below).


In [26]:
prompt = """Instruction: Provide long answers. 
Input: What is the ideal food for pregnant women?
Output: """
print(generate_response(prompt))

tion: Provide long answers. 
Input: What is the ideal food for pregnant women?
Output: The recommended dietary intake of folate during this period should be at least one mg/day, and ideally two to three times that amount (2-3mgs).


In [30]:
prompt = """Instruction: Provide long answers. 
Input: What are the additional diagnostic tests for lung cancer?
Output: """
print(generate_response(prompt))

tion: Provide long answers. 
Input: What are the additional diagnostic tests for lung cancer?
Output: Other imaging studies that may be helpful in diagnosing and staging of small cell carcinoma include positron emission tomography (PET) scan, bone scintigraphy with technetium-99m methoxine uptake study or fluorodexoyglucose PET/CT scan to detect metastasis into skeleton


In [31]:
prompt = """Instruction: Provide long answers. 
Input: What does thrombus grade 4 in TIMI mean?
Output: """
print(generate_response(prompt))

tion: Provide long answers. 
Input: What does thrombus grade 4 in TIMI mean?
Output: Thrombi are present at the site of occlusion and extend proximally into a major epicardial artery or distal coronary segment, but do not involve any other vessel segments (either within that same branching pattern as they did initially).


In [33]:
prompt = """Instruction: Provide long answers. 
Input: Can you provide a summary of upper gastrointestinal bleeding?

Output: """
print(generate_response(prompt))

tion: Provide long answers. 
Input: Can you provide a summary of upper gastrointestinal bleeding?

Output: Upper GI Bleed is defined as the passage through any part orifice in esophagus, stomach and duodenum (excluding rectal) with hematemesis that lasts for more than one hour without an obvious cause such as trauma to abdomen/chest wall /epigastric region; perforation from peptic ulcer disease ; Mallory-Weiss tear .
